### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Referirse a una columna

In [0]:
from pyspark.sql.functions import col, concat, lit
import datetime

usuario = [(1,
            'Corrie',
            'Van den Oord',
            'cvandenoord@etsy.com',
            True,
            1000.55, 
            datetime.date(2021, 1, 15), 
            datetime.datetime(2021, 2, 10, 1, 15)),
           (2,
            'Nikolas',
            'Brewitt',
            'nkbrewitt@gmail.com',
            True,
            900.0, 
            datetime.date(2021, 2, 14), 
            datetime.datetime(2021, 2, 18, 3, 33)),
           (3,
            'Oriel',
            'Penney',
            'openney@gmail.com',
            True,
            850.55, 
            datetime.date(2021, 1, 21), 
            datetime.datetime(2021, 3, 15, 15, 16, 55)),
           (4,
            'Ashby',
            'Maddocks',
            'amaddocks@gmail.com',
            False,
            None, 
            None, 
            datetime.datetime(2021, 4, 10, 17, 45, 30)),
           (5,
            'Kurt',
            'Rome',
            'krome@etsy.com',
            False,
            None, 
            None, 
            datetime.datetime(2021, 4, 2, 0, 55, 18))
          ]

usuario_schema = '''
    id INT,
    nombre STRING,
    apellido STRING,
    email STRING,
    es_cliente BOOLEAN,
    importe_abonado FLOAT,
    cliente_desde DATE,
    ultima_actualizacion TIMESTAMP
'''

df = spark.createDataFrame(usuario, schema=usuario_schema)

#### Forma 1

In [0]:
df.select('id', col('nombre'), 'apellido').show()

+---+-------+------------+
| id| nombre|    apellido|
+---+-------+------------+
|  1| Corrie|Van den Oord|
|  2|Nikolas|     Brewitt|
|  3|  Oriel|      Penney|
|  4|  Ashby|    Maddocks|
|  5|   Kurt|        Rome|
+---+-------+------------+



#### Forma 2

In [0]:
df.select(df['id'], col('nombre'), 'apellido').show()

+---+-------+------------+
| id| nombre|    apellido|
+---+-------+------------+
|  1| Corrie|Van den Oord|
|  2|Nikolas|     Brewitt|
|  3|  Oriel|      Penney|
|  4|  Ashby|    Maddocks|
|  5|   Kurt|        Rome|
+---+-------+------------+



#### Forma 3

In [0]:
df.alias('u')

Out[8]: DataFrame[id: int, nombre: string, apellido: string, email: string, es_cliente: boolean, importe_abonado: float, cliente_desde: date, ultima_actualizacion: timestamp]

In [0]:
df.alias('u').select(u['id'], col('nombre'), 'apellido').show()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-804623631765848> in <cell line: 1>()
----> 1 df.alias('u').select(u['id'], col('nombre'), 'apellido').show()

NameError: name 'u' is not defined

#### Forma 4

In [0]:
df.alias('u').select('u.id', col('nombre'), 'apellido').show()

+---+-------+------------+
| id| nombre|    apellido|
+---+-------+------------+
|  1| Corrie|Van den Oord|
|  2|Nikolas|     Brewitt|
|  3|  Oriel|      Penney|
|  4|  Ashby|    Maddocks|
|  5|   Kurt|        Rome|
+---+-------+------------+



#### Forma 5
Esto no funciona ya que selectExpr sólo puede tomar nombres de columnas o expresiones de estilo SQL sobre nombres de columnas

In [0]:
df.selectExpr(col('id'), 'nombre', 'apellido').show()

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<command-804623631765852> in <cell line: 1>()
----> 1 df.selectExpr(col('id'), 'nombre', 'apellido').show()

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/dataframe.py in selectExpr(self, *expr)
   2132         if len(expr) == 1 and isinstance(expr[0], list):
   2133             expr = expr[0]  # type: ignore[assignment]
-> 2134         jdf = self._jdf.selectExpr(self._jseq(expr))
   2135         return DataFrame(jdf, self.sparkSession)
   2136 

/databricks/spark/python/pyspark/sql/datafr

#### Forma 6

In [0]:
df.alias('u').selectExpr('id', 'nombre', 'apellido', "concat(u.nombre, ', ', u.apellido) AS nombre_completo").show(truncate=False)

+---+-------+------------+--------------------+
|id |nombre |apellido    |nombre_completo     |
+---+-------+------------+--------------------+
|1  |Corrie |Van den Oord|Corrie, Van den Oord|
|2  |Nikolas|Brewitt     |Nikolas, Brewitt    |
|3  |Oriel  |Penney      |Oriel, Penney       |
|4  |Ashby  |Maddocks    |Ashby, Maddocks     |
|5  |Kurt   |Rome        |Kurt, Rome          |
+---+-------+------------+--------------------+



#### Forma 7

In [0]:
df.select('id', 'nombre', 'apellido', concat(df['nombre'], lit(' '), col('apellido')).alias('nombre_completo')).show(truncate=False)

+---+-------+------------+-------------------+
|id |nombre |apellido    |nombre_completo    |
+---+-------+------------+-------------------+
|1  |Corrie |Van den Oord|Corrie Van den Oord|
|2  |Nikolas|Brewitt     |Nikolas Brewitt    |
|3  |Oriel  |Penney      |Oriel Penney       |
|4  |Ashby  |Maddocks    |Ashby Maddocks     |
|5  |Kurt   |Rome        |Kurt Rome          |
+---+-------+------------+-------------------+



#### Forma 8

In [0]:
df.createOrReplaceTempView('usuarios')

spark.sql('''
    SELECT id, nombre, apellido, concat(u.nombre, ', ', u.apellido) AS nombre_completo
    FROM usuarios AS u
    ''').show(truncate=False)

+---+-------+------------+--------------------+
|id |nombre |apellido    |nombre_completo     |
+---+-------+------------+--------------------+
|1  |Corrie |Van den Oord|Corrie, Van den Oord|
|2  |Nikolas|Brewitt     |Nikolas, Brewitt    |
|3  |Oriel  |Penney      |Oriel, Penney       |
|4  |Ashby  |Maddocks    |Ashby, Maddocks     |
|5  |Kurt   |Rome        |Kurt, Rome          |
+---+-------+------------+--------------------+



#### Forma 9

In [0]:
cols = ['id', 'nombre', 'apellido']

df.select(*cols).show(truncate=False)

+---+-------+------------+
|id |nombre |apellido    |
+---+-------+------------+
|1  |Corrie |Van den Oord|
|2  |Nikolas|Brewitt     |
|3  |Oriel  |Penney      |
|4  |Ashby  |Maddocks    |
|5  |Kurt   |Rome        |
+---+-------+------------+

